<a href="https://colab.research.google.com/github/dspauloplima/PySpark/blob/main/PySpark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PySpark SQL

In [2]:
!pip -q install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 40.1MB/s 


In [18]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import pandas_udf

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])

df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



## Tabela temporária

In [7]:
df.createTempView("tableA")

## SQL Queries

In [12]:
spark.sql("SELECT count(*) FROM tableA").show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [30]:
#função
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
  return s+1

# registra uma função python como uma função SQL
spark.udf.register("add_one", add_one)

#aplicação
spark.sql("SELECT add_one(v1) as Add_One FROM tableA LIMIT 3").show()

+-------+
|Add_One|
+-------+
|      2|
|      3|
|      4|
+-------+



In [40]:
from pyspark.sql.functions import expr

df.selectExpr("add_one(v1)").show()
df.select(expr("count(*)") > 5).show()
df.select(expr("count(*)") > 8).show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+

+--------------+
|(count(1) > 5)|
+--------------+
|          true|
+--------------+

+--------------+
|(count(1) > 8)|
+--------------+
|         false|
+--------------+

